<a href="https://colab.research.google.com/github/Bilal-Ahmad-5/Langchain-Intro/blob/main/self_reflective_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade langchain langgraph langchain_community langchain-google-genai langchain_chroma langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 743.8 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 11.2 MB/s eta 0:00:00

In [ ]:
from google.colab import userdata
import os
os.environ['LANGCHAIN_PROJECT'] = "self-reflective-rag"
os.environ["LANGCHAI_TRACING_V2"] = "true"
os.environ['LANGCHAIN_API_KEY'] = userdata.get('langchai_api_key')
google_api_key = userdata.get('Gemini_Api_Key')
os.environ['GOOGLE_API_KEY'] = google_api_key
os.environ["TAVILY_API_KEY"] = userdata.get('tavily_api_key')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI

#load
urls = ["https://python.langchain.com/docs/tutorials/graph/",
        "https://python.langchain.com/docs/how_to/#retrievers",
        "https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/"]

docs = [WebBaseLoader(url).load_and_split() for url in urls]
docs_split = [items for sublist in docs for items in sublist]

#split
spliting = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs_split = spliting.split_documents(docs_split)

#embedding
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

#vectorestore

vectorstore = Chroma.from_documents(docs_split, embeddings)



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from typing import TypedDict , Dict
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  keys: Dict[str , any]

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser , PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel , Field
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.tools import tool
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import PromptTemplate


#nodes

def retrieve(state):
  print("----RETRIEVE----")
  state_dict = state["keys"]
  question = state_dict["question"]
  documents = retriever.get_relevant_documents(question)
  return {"keys": {"documents": documents , "question": question}}

def generation(state):
  print("----GENERATION----")
  state_dict = state["keys"]
  question = state_dict["question"]
  documents = retriever.get_relevant_documents(question)

  #prompt
  prompt = hub.pull("rlm/rag-prompt")

  #llm
  llm = ChatGoogleGenerativeAI(
      model= "gemini-1.5-flash-8b"
  )
  rag_chain = prompt | llm | StrOutputParser()
  #generation
  generation = rag_chain.invoke({"context": documents , "question": question})
  return {"keys": {"generation": generation , "documents" : documents , "question": question}}

def grade_documents(state):
  print("----Check Relevance----")
  state_dict= state["keys"]
  question = state_dict["question"]
  documents = state_dict["documents"]

  class grade(BaseModel):
    """ check the relevance documents"""

    binary_score: str = Field(
        description=("Check binary score 'yes' or 'no' ")
    )

  #llm
  llm = ChatGoogleGenerativeAI(
    model= "gemini-1.5-flash-8b"
  )

  #prompt
  prompt = PromptTemplate(
      template= """You are a grader accessing relevance of retrieved documents to the user question.\n
      Here is the retrieved documnets:\n{context}.\n
      Here is the user Question:{question}.\n
      If the documents contain keyword(s) or semantic meaning relative to the user question ,grade them as relevant.\n
      Give a relevance score 'yes' or 'no' score for all documents to indicate that weather all documents are relevant.\n
      your response should be in json format:""",
      input_variables=["context" , "question"]
  )

  chain= prompt | llm.with_structured_output(grade,include_raw=True)

  search = "no"
  filtered_docs=[]
  for d in documents:
    score = chain.invoke({"context": d.page_content , "question": question})
    if isinstance(score, tuple) and len(score) > 0 and hasattr(score[0], 'binary_score'):
      if score[0].binary_score == "yes":
            filtered_docs.append(d)
      else:
            print("----Documents are not relevant")
            search = "yes"
    else:
        # If structured output parsing failed, print message and potentially log the issue
        print("----Could not parse relevance score, skipping document")
        # Optionally, you can log the raw 'score' value for debugging
  return {"keys": {"documents": filtered_docs , "question": question , "search": search}}

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import Document

def translate_query(state):
  print("----Translate Query----")
  state_dict = state["keys"]
  question = state_dict["question"]
  documents = state_dict["documents"]

  #llm
  llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-8b")

  #prompt
  prompt = PromptTemplate(
      template=""" You are generating question that is well optimized for retrieval.\n
      Look at the input and try to reason about underlying semantic intent / meanings.\n
      Here is the initial question:
      \n-------\n
      {question}
      \n-------\n
      Formulate an improved question:""",
      input_variables=["question"]
  )

  #chain
  chain= prompt | llm | StrOutputParser()
  new_question = chain.invoke({"question": question})
  return {"keys": {"question": new_question , "documents": documents}}

def web_search(state):
  print("----Web Search----")
  state_dict = state["keys"]
  question = state_dict["question"]
  documents = state_dict["documents"]

  tavily = TavilySearchResults(max_results=1)
  tavily_search= tavily.invoke(question)
  web_results = "\n".join([d["content"] for d in tavily_search])
  web_results = Document(page_content=web_results)
  documents.append(web_results)
  return {"keys": {"documents": documents , "question": question}}

def decide(state):
  print("----Decide----")
  state_dict = state["keys"]
  question = state_dict["question"]
  documents = state_dict["documents"]
  search = state_dict["search"]

  if search == "yes":
    print("----DECISION: Translate Query and Search the Web")
    return "translate"
  else:
    print("----DECISION: Generation")
    return "generation"



In [ ]:
from langgraph.graph import StateGraph ,END

graph = StateGraph(AgentState)

graph.add_node("retrieve" , retrieve) #retrieve
graph.add_node("generation", generation) #generation
graph.add_node("grade_documents", grade_documents) #grade_documents
graph.add_node("translate_query", translate_query) #translate
graph.add_node("web_search", web_search) #web_search


graph.set_entry_point("retrieve")
graph.add_edge("retrieve" , "grade_documents")
graph.add_conditional_edges(
    "grade_documents",
    decide,
    {
        "translate": "translate_query",
        "generation": "generation"
    }
)
graph.add_edge("translate_query", "web_search" )
graph.add_edge("web_search","generation")
graph.add_edge("generation", END)

app = graph.compile()

In [ ]:
inputs = "how to retrieve relevant documents from vectorestore using langgraph?"
for output in app.stream({"keys": {"question":inputs}},{"recursion_limit":150}):
  for key, value in output.items():
    print(f"{key}: {value}")

/tmp/ipython-input-201475541.py:16: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  documents = retriever.get_relevant_documents(question)


----RETRIEVE----
retrieve: {'keys': {'documents': [Document(metadata={'description': 'Here you‚Äôll find answers to ‚ÄúHow do I‚Ä¶.?‚Äù types of questions.', 'title': 'How-to guides | \uf8ffü¶úÔ∏è\uf8ffüîó LangChain', 'source': 'https://python.langchain.com/docs/how_to/#retrievers', 'language': 'en'}, page_content='How to: use a vector store to retrieve data\n\nRetrievers‚Äã\nRetrievers are responsible for taking a query and returning relevant documents.\n\nHow to: use a vector store to retrieve data\nHow to: generate multiple queries to retrieve data for\nHow to: use contextual compression to compress the data retrieved\nHow to: write a custom retriever class\nHow to: add similarity scores to retriever results\nHow to: combine the results from multiple retrievers\nHow to: reorder retrieved results to mitigate the "lost in the middle" effect\nHow to: generate multiple embeddings per document\nHow to: retrieve the whole document for a chunk\nHow to: generate metadata filters\nHow to: cr